### Формирование файлов daily_energy.csv, daily_weather.csv, weekdays.csv

In [1]:
# импортируем библиотеку os
import os
#импортируем библиотеку time
import time

In [2]:
#сформируем список файлов в папке daily_dataset
fnames = os.listdir('./data/daily_dataset')
#проверим
print(len(fnames), 'files')
fnames[:10]

112 files


['block_98.csv',
 'block_46.csv',
 'block_37.csv',
 'block_42.csv',
 'block_36.csv',
 'block_13.csv',
 'block_62.csv',
 'block_68.csv',
 'block_26.csv',
 'block_108.csv']

**Формируем файл daily_energy.csv**

In [3]:
#считываем наименование первого файла списка fnames
fname = fnames[0]
#сбрасываем счетчик обработанных файлов
files_cnt = 0
#словарь обработанных дат
de_days_dict = {}

#открываем объединенный файл на запись, а файл данных fname на чтение
with open('./data/daily_energy.csv', 'w') as dstfile:
    with open(os.path.join('./data/daily_dataset', fname), 'r') as srcfile:
        #считываем первую строку с наименованиями полей данных
        row = srcfile.readline()
        #преобразуем строку в список
        fields = row.strip().split(',')
        #записываем заголовочную строку в объединенный файл
        dstfile.write('{},{},{},{}\n'.format(fields[0], 'house_id', fields[1], fields[7]))
        
#открываем объединенный файл на добавление данных        
with open('./data/daily_energy.csv', 'a') as dstfile:        
    #перебираем наименования всех файлов в списке fnames и собираем все данные в один файл
    for fname in fnames:
        #увеличиваем счетчик обработанных файлов
        files_cnt += 1
        #формируем идентификатор дома
        house_id = fname[:-4]
        house_id = house_id[6:]
        #print('house_id = ', house_id)
        #открываем файл данных fname на чтение
        with open(os.path.join('./data/daily_dataset', fname), 'r') as srcfile:
            #пропускаем первую строку (заголовок)
            next(srcfile)
            #последовательно считываем остальные строки
            for row in srcfile:
                #преобразуем строку в список
                fields = row.strip().split(',')
                #отфильтруем записи с полным числом отсчетов датчика (за сутки должно быть 
                #48 получасовых отсчетов)
                if fields[5] == '48':
                    #записываем строку в объединенный файл
                    dstfile.write('{},{},{},{}\n'.format(fields[0], house_id, fields[1], fields[7]))
                    #добавляем поле даты в словарь
                    de_days_dict[fields[1]] = 0

#отображаем количество обработанных файлов
print(files_cnt,'files processed')        

112 files processed


**Формируем файл daily_weather.csv**  

Работа с уже сформированным файлом daily_weather.csv показала, что он содержит дубликаты нескольких дат.
Для устранения дубликатов дат в алгоритм было добавлено использование словаря day_dict для подавления повтора дат. Кроме того, оказалось, что в колонках одной строки, содержащие различные отметки времени, могут быть указаны разные даты. Для определения, к какой дате относить текущую строку файла daily_weather.csv частотный алгоритм выбора даты. Выбирается дата, наиболее часто встречающаяся в колонках с отметками времени.

In [4]:
#сбрасываем счетчик обработанных дней
days_cnt = 0
#словарь с обработанными датами
day_dict = {}
#словарь дат в текущей строке
date_dict = {}
#индексы полей с датами
date_idx = [1, 5, 9, 14, 18, 19, 23, 24, 27, 29, 30]

#открываем на запись (создаем) файл daily_weather.csv
with open('./data/daily_weather.csv', 'w') as dstfile:
    #записываем заголовочную строку с наименованиями полей данных
    dstfile.write('{},{},{},{}\n'.format('day', 'temperatureMax', 'temperatureMin', 'daylight'))
    #открываем на чтение файл с данными о погоде
    with open('./data/weather_daily_darksky.csv', 'r') as srcfile:
        #пропускаем первую строку (заголовок)
        next(srcfile)
        #последовательно считываем остальные строки
        for row in srcfile:
            #преобразуем строку в список
            fields = row.strip().split(',')
            #формируем поле day (дата)
            #очищаем словарь дат
            date_dict = {}
            #в цикле считаем сколько раз встретилась дата в полях с отметками времени
            for i in date_idx:
                day = fields[i][:10]
                if day in date_dict:
                    date_dict[day] = date_dict[day] + 1
                else:
                    date_dict[day] = 1
            #ищем максимальное значение в словаре, ему будет соответствовать наиболее вероятная дата данных
            max_val = 0
            for k in date_dict:
                if date_dict[k] > max_val:
                    max_val = date_dict[k]
                    day = k
                    print(day, max_val)
       
            #если даты day нет в словаре day_dict - добавляем данные в файл
            if not (day in day_dict):
                #считываем строку времени захода солнца
                sunsetTimeStr = fields[19][11:]
                #преобразуем строку времени в объект struct_time 
                sunsetTime = time.strptime(sunsetTimeStr, '%H:%M:%S')
                #считываем строку времени восхода солнца
                sunriseTimeStr = fields[23][11:]
                #преобразуем строку времени в объект struct_time 
                sunriseTime = time.strptime(sunriseTimeStr, '%H:%M:%S')
                #формируем поле daylight (в секундах)
                daylight = time.mktime(sunsetTime) - time.mktime(sunriseTime)
                #записываем строку в целевой файл
                dstfile.write('{},{},{},{}\n'.format(day, fields[0], fields[21], round(daylight)))
                #записываем дату day в словарь
                day_dict[day] = 0;
                #увеличиваем счетчик дней
                days_cnt += 1
            else:
                print(day, 'duplicated!')
                
#отображаем количество обработанных дней 
print(days_cnt,'days processed')            

2011-11-11 11
2011-12-11 9
2011-12-27 10
2011-12-02 11
2011-12-24 11
2011-12-15 9
2011-11-19 9
2011-11-16 11
2011-12-12 9
2011-11-20 9
2011-11-08 11
2011-12-04 9
2011-12-16 9
2011-11-01 9
2011-12-26 10
2011-11-28 11
2011-12-09 9
2011-12-05 9
2011-11-24 11
2011-11-17 10
2011-12-13 10
2011-12-22 11
2011-12-06 11
2011-12-07 9
2011-11-21 11
2011-12-08 9
2011-11-06 9
2011-11-25 9
2011-12-01 9
2011-12-25 11
2011-12-30 11
2011-11-04 9
2011-11-10 9
2011-11-09 9
2011-11-03 9
2011-11-07 9
2011-11-23 11
2011-11-18 9
2011-12-21 9
2011-12-19 9
2011-12-28 9
2011-12-31 9
2011-12-18 9
2011-11-15 9
2011-12-23 9
2011-11-26 11
2011-11-05 9
2011-11-12 9
2011-12-10 9
2011-11-29 9
2011-11-13 9
2011-11-22 9
2011-12-14 9
2011-12-03 9
2011-12-17 9
2011-12-29 9
2011-11-27 9
2011-11-30 9
2011-11-14 9
2011-12-20 10
2011-11-02 11
2012-11-25 9
2012-05-19 8
2012-07-14 8
2012-09-23 10
2012-05-26 8
2012-01-06 11
2012-11-20 9
2012-05-02 2
2012-05-03 7
2012-04-19 8
2012-05-05 8
2012-04-14 8
2012-11-01 11
2012-05-25 8
20

2013-08-12 8
2013-12-24 9
2013-01-16 9
2013-07-13 8
2013-11-17 9
2013-05-03 8
2013-08-25 8
2013-03-27 9
2013-06-14 8
2013-04-13 10
2013-08-15 6
2013-07-25 8
2013-01-31 9
2013-03-23 9
2013-11-24 9
2013-10-16 8
2013-11-07 9
2013-10-18 8
2013-07-30 8
2013-01-28 9
2013-12-05 9
2013-04-07 8
2013-05-30 8
2013-07-29 8
2013-04-03 8
2013-07-17 8
2013-08-01 8
2013-02-09 11
2013-01-26 11
2013-12-29 9
2013-08-19 8
2013-07-03 8
2013-02-27 10
2013-03-09 9
2013-07-06 8
2013-12-30 11
2013-03-13 9
2013-12-03 11
2013-06-18 9
2013-01-05 9
2013-06-02 8
2013-09-03 8
2013-09-08 8
2013-08-05 8
2013-02-05 9
2013-08-23 8
2013-03-21 9
2013-10-06 8
2013-11-06 9
2013-04-17 8
2013-01-30 9
2013-12-27 9
2013-01-06 9
2013-11-20 9
2013-07-28 8
2013-01-25 11
2013-06-27 10
2013-06-09 8
2013-11-23 9
2013-10-09 8
2013-09-18 8
2013-01-21 9
2013-10-15 8
2013-07-22 8
2013-05-24 8
2013-12-10 9
2013-10-01 8
2013-11-14 9
2013-06-25 8
2013-02-23 9
2013-09-08 3
2013-09-09 6
2013-12-28 9
2014-01-12 9
2014-01-24 11
2014-01-14 11
20

In [5]:
#проверяем, что для каждой даты в файле daily_energy есть данные о погоде в файле daily_weather.csv
absent_cnt = 0
for k in de_days_dict:
    if not (k in day_dict):
        print(k, 'is absent!')
        absent_cnt += 1
        
print(absent_cnt, 'date(s) absent')

0 date(s) absent


**Формируем файл weekdays.csv (номера дней недели в стандарте ISO)**

In [6]:
#импортируем библиотеку datetime
from datetime import datetime as dt

In [7]:
#сбрасываем счетчик обработанных дней
days_cnt = 0

#открываем на запись (создаем) файл weekdays.csv
with open('./data/weekdays.csv', 'w') as dstfile:
    #записываем заголовочную строку с наименованиями полей данных
    dstfile.write('{},{}\n'.format('day', 'isoweekday'))
    #создаем отсортированный список дат из словаря day_dict
    day_list = sorted(day_dict)
    #пробегаем отсортированный список дат и формируем запись в weekday.csv
    for day in day_list:
        #преобразуем строку в формат datetime
        day_dt = dt.strptime(day, '%Y-%m-%d')
        #преобразуем в номер дня недели (по стандарту ISO: Пн - 1 .. Вс - 7)
        weekday = day_dt.isoweekday()   #strftime("%a")
        #записываем в файл
        dstfile.write('{},{}\n'.format(day, weekday))
        #увеличиваем счетчик обработанных дней
        days_cnt += 1
        
#отображаем количество обработанных дней 
print(days_cnt,'days processed')

882 days processed
